In [1]:
# setup dataset
from datasets import Dataset
import numpy as np
import faiss
retrieval_vector_size = 768

dataset = Dataset.from_dict(
    {
        "id": ["0", "1"] * 5,
        "text": ["My favourite number is 3455", "The secret word is FROG"] * 5,
        "embeddings": [
            0.1 * np.ones(retrieval_vector_size),
            0.9 * np.ones(retrieval_vector_size),
        ] * 5,
    }
)

dataset.add_faiss_index("embeddings", metric_type=faiss.METRIC_INNER_PRODUCT)


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:00<00:00, 704.21it/s]


Dataset({
    features: ['id', 'text', 'embeddings'],
    num_rows: 10
})

In [2]:
# query_hidden_states = query_hidden_states.cpu().detach().numpy()
# _, passage_ids = self.index.search_batch("embeddings", query_hidden_states, topk)
# docs = [self.index[[i for i in indices if i >= 0]] for indices in passage_ids]

# # todo: move this out to retriever - it's not the model's job to do this and it should be configurable
# passages = [[f'{queries[i]} context: {passage}' for passage in doc["text"]] for i, doc in enumerate(docs)]


# def encode_passages(batch, tokenizer, max_length):
#     bsz = len(batch)
#     n = max([len(example) for example in batch])
#     batch = [example + [""] *  (n - len(example)) for example in batch]
#     batch = reduce(lambda a, b: a + b, batch)
#     tokens = tokenizer(
#         batch,
#         padding=True,
#         max_length=max_length,
#         return_tensors="pt",
#         truncation=True,
#     )
#     tokens = {k: v.view(bsz, n, -1) for k, v in tokens.items()}
    
#     return tokens

# reader_tokens = encode_passages(passages, self.generator_tokenizer, 512)

# labels = self.generator_tokenizer(target, return_tensors="pt", padding=True, truncation=True, max_length=512)['input_ids'].to(self.device)
# labels[labels == self.generator_tokenizer.pad_token_id] = -100

# reader_ids = reader_tokens["input_ids"].to(self.device)  # FIXME (Not added by ae99, TODO figure out why)
# reader_mask = reader_tokens["attention_mask"].bool().to(self.device)


# n_context_training = min(topk, reader_ids.size(1))
# cfg = self.generator.encoder.config
# cfg.bsz = reader_ids.size(0)
# cfg.n_context = n_context_training

# reader_ids_training = reader_ids[:, :n_context_training].contiguous()
# reader_mask_training = reader_mask[:, :n_context_training].contiguous()

# reader_ids_training = reader_ids_training.view(reader_ids.size(0), -1)
# reader_mask_training = reader_mask_training.view(reader_mask.size(0), -1)


In [3]:
from src.transformers.models.atlas.retrieval_atlas import AtlasRetrieverIndex, AtlasConfig, AtlasTokenizer

config = AtlasConfig.from_pretrained("./data/atlas-pretrained")
tokenizer = AtlasTokenizer.from_pretrained("./data/atlas-pretrained", config=config)

retrieverIndex = AtlasRetrieverIndex(config, tokenizer, dataset)

In [4]:
retriever_hidden_states = np.zeros((2, 768))
retriever_hidden_states

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
dataset.search_batch("embeddings", retriever_hidden_states, 1)

: 

: 

In [ ]:
retriever_hidden_states = np.ones((1, 768))
generator_input_ids = tokenizer.generator("hello world", return_tensors="pt")["input_ids"]

retrieverIndex(generator_input_ids, retriever_hidden_states)




AttributeError: 'Tensor' object has no attribute 'flags'